## Chat with PDF by MD ARHAN MANSOORI

In [5]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

local_path = "ARHAN ATS RESUME.pdf"
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")



data[0].page_content



text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)



vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="llama3.1", show_progress=True),
    collection_name="vector_collection",
    persist_directory="runs"
)



from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever


local_model = "llama3.1"
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


chain.invoke("what is this about?")

ValueError: Error raised by inference API HTTP code: 500, {"error":"model requires more system memory (5.9 GiB) than is available (4.8 GiB)"}

In [3]:
local_path = "talib_resume_resumeword_color.pdf"
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [4]:

data[0].page_content

'The Story of the Trojan war\n\n2\n\nThe story of the Trojan war\n\nLong ago, when the gods still walked the earth, a great feast was held to celebrate the wedding of Thetis and Peleus. All the gods were invited except Eris, the goddess of quarrels. The wedding feast was almost over when Eris stormed in. She threw a golden apple on the table. On it were written the words: “For the fairest.” Each of the goddesses thought she was the fairest and should have the apple. All of them were very beautiful. Nobody could decide between them. They argued about it for years. At last the gods chose the three loveliest, and they decided to ask a human to judge between them. They chose Paris, the son of the king of Troy. He was very surprised one evening to see Hermes appear with three goddesses. They were: Hera, the queen of the gods; Athena, the goddess of wisdom; and Aphrodite, the goddess of love and beauty. “Congratulations, Paris!” said Hermes. “You have been chosen to judge which goddess deser

## generating vector embeddings using llama3.1

In [5]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕████████████████▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [6]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
llama3.1:latest        	91ab477bec9d	4.7 GB	Less than a second ago	
nomic-embed-text:latest	0a109f422b47	274 MB	3 days ago            	
llama3:latest          	365c0bd3c000	4.7 GB	8 days ago            	
mistral:latest         	f974a74358d6	4.1 GB	10 days ago           	


In [7]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [9]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [10]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="llama3.1", show_progress=True),
    collection_name="vector_collection",
    persist_directory="runs"
)

OllamaEmbeddings: 100%|██████████| 1/1 [07:26<00:00, 446.60s/it]


## Retrieval from the vector database 

In [11]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever


local_model = "mistral"
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


chain.invoke("what is this about?")

In [12]:

local_model = "mistral"
llm = ChatOllama(model=local_model)

In [13]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [14]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke("what is this about?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:12<00:00, 12.35s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.59s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:06<00:00,  6.09s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.89s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.86s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' The provided text is about the story of the Trojan War. It describes how the war started due to a dispute between three goddesses (Hera, Athena, and Aphrodite) over a golden apple given as a prize for beauty. Paris, the prince of Troy, was asked to judge the fairest among them, which eventually led to the abduction of Helen, the most beautiful woman in Greece, by Paris. The Greeks then sailed to Troy to retrieve her, resulting in a prolonged war. Characters like Achilles, Hector, and other heroes are also mentioned, highlighting the events and consequences of the Trojan War.'